In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [3]:
Options = Options()
Options.add_argument('--headless')
driver = webdriver.Chrome(options=Options)

In [4]:
url_base = f"https://www.pagina12.com.ar"

In [5]:
soup = []
for i in range(0,2):
    print(i, end='\r')
    driver.get(f"{url_base}/buscar?q=milei&page={i}")
    pag =  driver.page_source
    soup.append(BeautifulSoup(pag, 'html5lib'))

In [6]:
paginas = []
for i in range(0,2):
    paginas.append(soup[i].find_all('div', class_='article-item__content'))

In [7]:
noticias = pd.DataFrame(columns=['indice','titulo','fecha','link','source','contenido','copete'])

i = -1
for pagina in paginas:
    for noticia in pagina:
        i = i + 1
        noticias.loc[i,'indice'] = i
        noticias.loc[i,'titulo'] = (noticia.get_text())
        noticias.loc[i,'link'] = f"{url_base}{noticia.find('a')['href']}"
        print(f'Procesado articulo {i}', end='\r')
        #print(f"Articulo: {i} Titulo: \n{titulo}\nLink:\n{link}\n-------------------------------------------------------------------------\n")


In [8]:
for index, row in noticias.iterrows():
    print(f"Articulo {row.indice} - {row.link}", end='\r')
    driver.get(row.link)
    pag =  driver.page_source
    row.source = BeautifulSoup(pag, 'html5lib')

In [9]:
for index, row in noticias.iterrows():
    #print(f"Articulo {row.indice} - {row.link}")
    contenido_source = row.source.find('div', class_='section-2-col article-main-content')
    row.contenido = contenido_source.get_text()
    copete_source = row.source.find('div', class_='section-2-col article-header')
    row.copete = copete_source.get_text()
    fecha_source = row.source.find('meta', property='article:modified_time')
    fecha = int(fecha_source.get('content'))
    fecha = datetime.datetime.fromtimestamp(fecha)
    row.fecha = fecha

In [10]:
noticias.to_excel('pagina12.xlsx')

In [11]:
driver.close()